In [2]:
1973/22.5

87.68888888888888

In [3]:
75*3+89*1.5+90*3+93*3+95*4+99*1+86*3+92*1+88*3+85*1`

2085.5

In [4]:
2085.5/23.5

88.74468085106383

In [5]:
88*3+92*1.5+86*3+97*3+91*4+99*1+81*3+88*1+82*3+89*1

2080.0

In [6]:
2080/23.5

88.51063829787235

In [7]:
78*3+90*1.5+81*3+98*3+94*4+93*3+86*1+79*3+87*1

1971.0

In [8]:
1971/22.5

87.6

In [25]:
from math import *


k=0
class cnt(object):
    def __init__(self,x1,x2):
        self.k=0
        self.x1=x1
        self.x2=x2
    def f(self,x):
        y=pow(x,3)-2*x-5
        print (y)
    def ff(self,x,y):
        print((x+y)/2)
    
    def func(self):
        self.k+=1
        x0=float((self.x1+self.x2)/2)
#         print(x0)
        y0=self.f(x0)
#         print(y0)
        if y0*self.f(self.x1)<0:
            self.x2=x0
        elif y0*self.f(self.x2)<0:
            self.x1=x0
        self.func()
        print("{} y={}".format(k,y0))
        if(self.k>10):
            return 0

cnt(2,3).f(2.0947265625)
cnt(2,3).ff(2.09375,2.095703125)
2.0947265625-2.095703125

0.00195434782654047
2.0947265625


-0.0009765625

In [1]:
import math


0.013888888888888888